# Explaining text classifications 


See [Transformers Interpret Multiclass Classification Example](https://github.com/cdpierse/transformers-interpret/blob/master/notebooks/multiclass_classification_example.ipynb)

In [ ]:
# Adds multiclass_classification.pkz to sys.path which includes a captum entry_point for explaining integrated gradients. See multclass_classification.yaml
# To create the multiclass_classification.pkz run 'explainer export multiclass_classification'

from explainer.explainers import lm_classifier_explainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

## Import Industry Classification Mode
This finetuned model by @sampathkethineedi uses a distilbert base to predict the professional industry a text is referring to. 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sampathkethineedi/industry-classification")
model = AutoModelForSequenceClassification.from_pretrained("sampathkethineedi/industry-classification")

Let's explore the classes, there are 62 unique classes. Many of these are overlapping/related industries such as __Health Care Equipment__ and __Health Care Supplies__.

In [ ]:
model.config.id2label

Import __SequenceClassificationExplainer__ from transformers interpret. This class should work with most if not all language models with a sequence classification head from the transformers package. 


In [ ]:
sample_text = """
Stocks ended a choppy session mixed as investors digested a host of corporate earnings results and considered policymakers’ next moves to support the still virus-stricken economy.
The S&P 500 shook off earlier declines to narrowly eke out a record closing high.The Dow ended a tick below its recent record closing level."""

In [ ]:
multiclass_explainer = lm_classifier_explainer['classify'](model,tokenizer)

In [ ]:
# call the exlplainer
word_attributions = multiclass_explainer(text=sample_text)

In [ ]:
# seems to be an appropriate prediction
multiclass_explainer.predicted_class_name

In [ ]:
# look the the raw word attributions
word_attributions

## Visualizating Explanations 
With a single call to the `visualize()` method we get a nice inline display of what inputs are causing the activations to fire that led to this prediction. **Note the alogirthm used to calcualte attributions are Layer Integreated Gradients to read more about them click [here](https://captum.ai/docs/algorithms)**

In [ ]:
html = multiclass_explainer.visualize()

## Explaining The Same Text For A Different Class 
Lets say we think this text could also fall somewhat under the class of __Asset Management & Custody Banks__ If we want it is also possible to get an explantion/attributions for the text with that class

In [ ]:
word_attributions = multiclass_explainer(sample_text, class_name="Asset Management & Custody Banks")

In [ ]:
# look the the raw word attributions
word_attributions

The results are close to the first visualization, a good sign that the model is generalizing well for both of these related classes

In [ ]:
html = multiclass_explainer.visualize()

What if we get attributions for class that makes no sense in this context such as __Restaurants__?


In [ ]:
word_attributions = multiclass_explainer(sample_text, class_name="Restaurants")

There isn't much to this prediction, it is worth nothing however that the words "choppy" had a more positive impact in this instance which seems plausible given the industry. 

In [ ]:
html = multiclass_explainer.visualize()

In [ ]:
word_attributions